# 환경 설정

In [ ]:
!pip install transformers

In [ ]:
import os
data_path = "drive/MyDrive/2025/KW/Data/감정 분류를 위한 대화 음성 데이터셋/"
os.listdir(data_path)

['5차년도.csv', '5차년도_2차.csv', '4차년도.csv']

In [ ]:
# Seed 고정
import random
import numpy as np
import torch
import os

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # multi-gpu

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(42)

# 데이터 로드

In [ ]:
import pandas as pd
df = pd.read_csv(data_path + "5차년도_2차.csv", encoding="cp949")

# Label Encoding
label_map = {
    "fear": 0,
    "surprise": 1,
    "angry": 2,
    "sadness": 3,
    "neutral": 4,
    "happiness": 5,
    "disgust": 6
}
df["y"] = df["상황"].map(label_map)

x_col = '발화문'
y_col = 'y'
input_data = df[[x_col] + [y_col]]
input_data

,발화문,y
0,헐! 나 이벤트에 당첨 됐어.,5
1,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,5
2,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",5
3,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",5
4,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,4
...,...,...
19369,나 엘리베이터에 갇혔어.,0
19370,하지만 기분이 나쁜 걸 어떡해?,2
19371,자취방 엘리베이턴데 정전인가봐.,0
19372,나 드디어 프로젝트 끝났어!,5


Split

In [ ]:
from sklearn.model_selection import train_test_split
trval_X, test_X, trval_y, test_y = train_test_split(
    input_data[x_col].tolist(), input_data[y_col].tolist(),
    test_size=0.05, stratify=input_data[y_col], random_state=42)

from sklearn.model_selection import train_test_split
train_X, valid_X, train_y, valid_y = train_test_split(
    trval_X, trval_y, test_size=0.05, stratify=trval_y, random_state=42)

print(f"            x      y")
print(f"train size: {len(train_X):<5}  {len(train_y):<5}")
print(f"valid size: {len(valid_X):<5}  {len(valid_y):<5}")
print(f"test size : {len(test_X):<5}  {len(test_y):<5}")

            x      y
train size: 17484  17484
valid size: 921    921  
test size : 969    969  


# 모델



In [ ]:
model_path = "drive/MyDrive/2025/KW/Model/"
model_id = "monologg/kobert"

## Input

In [ ]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=model_path, trust_remote_code=True)

In [ ]:
class CustomKoBERTTokenizer:
    def __init__(self, vocab_size=1000):
        pass

In [ ]:
# 토크나이저 Train
corpus = input_data['발화문'].tolist()
custom_tokenizer = CustomKoBERTTokenizer(vocab_size=5000)
custom_tokenizer.train(corpus)

In [ ]:
print("target vocab_size:", custom_tokenizer.vocab_size)
print("actual tokens   :", len(custom_tokenizer.token_to_id))  # 실제 vocab size

target vocab_size: 5000
actual tokens   : 4702


In [ ]:
from torch.utils.data import Dataset, DataLoader

class KoBERTDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        inputs = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            max_length=self.max_len
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
train_dataset = KoBERTDataset(train_X, train_y, custom_tokenizer)
valid_dataset = KoBERTDataset(valid_X, valid_y, custom_tokenizer)
test_dataset = KoBERTDataset(test_X, test_y, custom_tokenizer)

In [ ]:
from torch.utils.data import Dataset, DataLoader
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Model

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import BertModel
bert = BertModel.from_pretrained(model_id, cache_dir=model_path, trust_remote_code=True)

In [ ]:
import torch.nn as nn

class KoBERTClassifier(nn.Module):
    def __init__(self, bert, num_classes, hidden_size=768, dropout=0.2):
        super(KoBERTClassifier, self).__init__()
        self.bert = bert
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.pooler_output
        dropped = self.dropout(pooled)
        return self.classifier(dropped)

In [ ]:
num_classes = len(df['y'].unique())
num_classes  # 7

7

In [ ]:
model = KoBERTClassifier(bert, num_classes=num_classes).to(device)

## Train


In [ ]:
from torch.optim import AdamW
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score

# Optimizer and loss
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

epochs = 3

for epoch in range(epochs):
    # Training
    model.train()
    train_loss = 0
    train_preds = []
    train_labels = []

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} - Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

    train_loss = train_loss / len(train_loader)
    train_acc = accuracy_score(train_labels, train_preds)

    # Validation
    model.eval()
    valid_loss = 0
    valid_preds = []
    valid_labels = []

    with torch.no_grad():
        for batch in valid_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)

            valid_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            valid_preds.extend(preds.cpu().numpy())
            valid_labels.extend(labels.cpu().numpy())

    valid_loss = valid_loss / len(valid_loader)
    valid_acc = accuracy_score(valid_labels, valid_preds)

    print(f"Epoch {epoch+1} - loss: {train_loss:.4f}  acc: {train_acc:.4f} | "
          f"val loss: {valid_loss:.4f}  val acc: {valid_acc:.4f}")

Epoch 1 - Training:   0%|          | 0/274 [00:00<?, ?it/s]

Epoch 1 - loss: 1.6938  acc: 0.3444 | val loss: 1.3571  val acc: 0.5353


Epoch 2 - Training:   0%|          | 0/274 [00:00<?, ?it/s]

Epoch 2 - loss: 1.1505  acc: 0.5967 | val loss: 0.9266  val acc: 0.6699


Epoch 3 - Training:   0%|          | 0/274 [00:00<?, ?it/s]

Epoch 3 - loss: 0.8436  acc: 0.7106 | val loss: 0.7956  val acc: 0.7362


## Evaluation

In [ ]:
# Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {acc:.4f}")

Test Accuracy: 0.7647
